# 1. Introduction

Nikki Satmaka - Batch 11

## Description

Dataset is taken from [Google Cloud Platform BigQuery](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ml_datasets&t=credit_card_default&page=table)

Context:

This dataset contains 

### Objective

- pass

### Problem Statement

- pass

### Data Collection

We first need to query our data from GCP's BiqQuery using this code

```SQL
SELECT *
FROM table
```

# 2. Importing Libraries

In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from pathlib import Path

from pandas_profiling import ProfileReport

import joblib

import warnings
warnings.filterwarnings('ignore')

# For Handling outlier
from feature_engine.outliers import OutlierTrimmer
from feature_engine.outliers import Winsorizer
from feature_engine.outliers import ArbitraryOutlierCapper

# For Feature Engineering
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

# For Handling Missing Values
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import RandomSampleImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.imputation import ArbitraryNumberImputer

# For Data Preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# For Classification Problems
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier

# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV

# Split Dataset and Standarize the Datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler

# Evaluate Classification Models
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import auc


pd.set_option('display.precision', 2)

sns.set_theme(style='darkgrid', palette='deep')

%matplotlib inline

### Useful Functions

In [2]:
def check_unique(data, col_type='both'):
    """
    Count the number of unique values in each features for 'numeric', 'categorical', or 'both'

    Parameters
    ----------
    data : DataFrame

    col_type : str
        The type of the column to filter. Either 'number', 'object', or 'both'

    Returns
    -------
    DataFrame
        Number of unique values of each features
    """

    # check if the column type is valid
    if col_type not in ('number', 'object', 'both'):
        raise ValueError('col_type must be either "number", "object", or "both"')

    # create a list if the column type is 'both'
    if col_type == 'both':
        col_type = ['number', 'object']

    # get the number of unique values in each column
    data_unique_count = pd.DataFrame.from_records(
        [(col, data[col].nunique()) for col in data.select_dtypes(include=col_type).columns],
        columns=['feats', 'num_unique']
    )

    return data_unique_count

In [3]:
def find_normal_boundaries(data, variable):
    """
    Calculate the boundaries outside which sit the outliers for a Gaussian distribution

    Parameters
    ----------
    data : DataFrame

    variable : string
        The feature of the DataFrame in which to the calculation will be performed

    Returns
    -------
    upper_boundary : float
        The computed upper boundary of the data

    lower_boundary : float
        The computed lower boundary of the data
    """

    upper_boundary = data[variable].mean() + 3 * data[variable].std()
    lower_boundary = data[variable].mean() - 3 * data[variable].std()

    return upper_boundary, lower_boundary

In [4]:
def find_skewed_boundaries(data, variable, distance):
    """
    Calculate the boundaries outside which sit the outliers for skewed distribution

    Parameters
    ----------
    data : DataFrame

    variable : string
        The feature of the DataFrame in which to the calculation will be performed

    distance : float
        The distance multiplier of IQR to calculate the boundaries

    Returns
    -------
    upper_boundary : float
        The computed upper boundary of the data

    lower_boundary : float
        The computed lower boundary of the data
    """

    IQR = data[variable].quantile(0.75) - data[variable].quantile(0.25)

    upper_boundary = data[variable].quantile(0.75) + (IQR * distance)
    lower_boundary = data[variable].quantile(0.25) - (IQR * distance)

    return upper_boundary, lower_boundary

In [5]:
def check_dist(data):
    """
    Check the Skewness and Distribution for each features in a dataset

    Parameters
    ----------
    data : DataFrame

    Returns
    -------
    DataFrame
        Skewness and distribution types of each features
    """

    # create a DataFrame containing the features of the dataset and their respective skewness
    data_skewness = pd.DataFrame(data.skew(), columns=['skew']).reset_index()

    # reset the index and make the features columns
    data_skewness = data_skewness.rename(columns={'index': 'feats'})

    # create a new column to describe whether the feature in the dataset is normal or skewed
    data_skewness['dist'] = np.where(
        (data_skewness['skew'] > -0.5) & (data_skewness['skew'] < 0.5),
        'normal',
        'skewed'
    )

    return data_skewness

In [6]:
def check_outlier(data, distance=1.5):
    """
    Check the outlier info for each features in a dataset

    Parameters
    ----------
    data : DataFrame

    distance : float
        The distance multiplier of IQR to calculate the boundaries for skewed distributions. It's either 1.5 or 3

    Returns
    -------
    DataFrame
        Outlier infos such as upper and lower boundary, and also the number of outliers for each features
    """

    if distance not in (1.5, 3):
        raise ValueError('Parameter distance only accepts numeric value of either 1.5 or 3')

    data_skewness = check_dist(data)

    # create a dictionary to store the outlier infos
    data_outlier = {
        'feats': [],
        'upper_bound': [],
        'lower_bound': [],
        'tot_right_tail': [],
        'tot_left_tail': [],
        'tot_right_tail_pct': [],
        'tot_left_tail_pct': [],
        'tot_outlier': [],
        'tot_outlier_pct': [],
    }

    # loop over each row in the `skewness` DataFrame
    # calculate each features upper and lower boundaries and the outlier percentage
    for row in data_skewness.index:
        col = data_skewness.iloc[row]['feats']

        if data_skewness.iloc[row]['dist'] == 'normal':
            upper_bound, lower_bound = find_normal_boundaries(data, col)
        else:
            upper_bound, lower_bound = find_skewed_boundaries(data, col, distance)

        tot_right_tail = len(data[data[col] > upper_bound])
        tot_left_tail = len(data[data[col] < lower_bound])
        tot_right_tail_pct = tot_right_tail / len(data) * 100
        tot_left_tail_pct = tot_left_tail / len(data) * 100
        tot_outlier =  tot_right_tail + tot_left_tail
        tot_outlier_pct = tot_right_tail_pct + tot_left_tail_pct

        data_outlier['feats'].append(col)
        data_outlier['upper_bound'].append(upper_bound)
        data_outlier['lower_bound'].append(lower_bound)
        data_outlier['tot_right_tail'].append(tot_right_tail)
        data_outlier['tot_left_tail'].append(tot_left_tail)
        data_outlier['tot_right_tail_pct'].append(tot_right_tail_pct)
        data_outlier['tot_left_tail_pct'].append(tot_left_tail_pct)
        data_outlier['tot_outlier'].append(tot_outlier)
        data_outlier['tot_outlier_pct'].append(tot_outlier_pct)
    
    data_outlier = pd.DataFrame(data_outlier)

    return data_outlier

In [7]:
def outlier_summary(data, distance=1.5):
    """
    Check the summary for outlier data, such as distribution and number of outliers for each features

    Parameters
    ----------
    data : DataFrame

    distance : float
        The distance multiplier of IQR to calculate the boundaries for skewed distributions. It's either 1.5 or 3

    Returns
    -------
    DataFrame
        Summary of outlier such as distribution and number of outliers for each features
    """

    data_skewness = check_dist(data)
    data_outlier = check_outlier(data, distance)

    outlier_summary_cols = ['feats', 'skew', 'dist', 'tot_outlier', 'tot_outlier_pct']

    data_outlier_summary = pd.merge(data_skewness, data_outlier, on=['feats'])
    data_outlier_summary = data_outlier_summary[outlier_summary_cols]

    return data_outlier_summary

In [8]:
def impute_na(data, variable, mean_value, median_value):
  """
  Function to Fill Missing Values with Zeroes, Mean, and Median
  """
  data[variable+'_mean'] = data[variable].fillna(mean_value)
  data[variable+'_median'] = data[variable].fillna(median_value)
  data[variable+'_zero'] = data[variable].fillna(0)
  
  return data

# 3. Data Loading

In [ ]:
# load dataset
df_ori = pd.read_csv('data/')
df = df_ori.copy()

# display the first 5 entries of the data
df.head()

In [ ]:
# display the last 5 entries of the data
df.tail()

## Data Understanding

In [ ]:
# check dataset shape
df.shape

There are x entries and y columns of data

In [ ]:
# check dataset info
df.info()

In [ ]:
# check missing values in dataset
df.isna().sum().sort_values(ascending=False)

Great there are no missing values

## Basic Characteristics of the Dataset

In [ ]:
# check basic stats for features with number dtypes
df.describe(percentiles=[0.5]).T

In [ ]:
# check basic stats for features with object dtypes
df.describe(include='object').T

In [ ]:
# check the cardinality of each features
print("Features with low cardinality:")
for col in df.columns:
    if df[col].nunique() < 20:
        print(col, ':', df[col].nunique(), 'unique values \n', np.sort(df[col].unique()))
        print('-' * 100)

Let's cover some basic stats of some numerical features in the train set.
- `feature`
    - pass

## Data Preparation

Handle cardinality in `feature`

Data looks good and is in accordance with the data design

## Check for Dataset Imbalance

Check whether the target variable of the dataset is balance

In [ ]:
# check for imbalance in target variable
df['target'].value_counts().plot(kind='bar')
plt.show()

Since our dataset is imbalance, we need to stratify when splitting

## Splitting Dataset

We need to split the dataset into inference, train and test sets before we do any EDA.\
We do our EDA on the train set so as to not have any bias towards the whole dataset.

### Sample data for inference

In [ ]:
# sample dataset for inference
df_inf = df.sample(10, random_state=42)

# remove inference set from original dataset
df_train_test = df.drop(df_inf.index).reset_index(drop=True)

# reset index for inference set
df_inf = df_inf.reset_index(drop=True)

print('df_inf Size:', df_inf.shape)

### Split train and test set


Since the target variable is imbalanced, we use stratified sampling

In [ ]:
# we use stratified sampling to ensure that the distribution of the target variable is balanced
df_train, df_test = train_test_split(
    df_train_test,
    test_size=0.20,
    random_state=42,
    stratify=df_train_test['target']
)

print('df_train Size:', df_train.shape)
print('df_test Size:', df_test.shape)

In [22]:
# backup the train set that we are gonna perform EDA on
df_train_ori = df_train.copy()

# 4. Exploratory Data Analysis

## Source of dataset

The dataset seems to have come from [UCI](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)

## Subheading 1

## Subheading 2

## Subheading 3

# 5. Data Preprocessing

In [37]:
# restore the train set from the backup
df_train = df_train_ori.copy()

In [38]:
# split between features and target
X_train = df_train.drop(['target'], axis=1)
y_train = df_train['target'].copy()

X_test = df_test.drop(['target'], axis=1)
y_test = df_test['target'].copy()

## Categorizing Features

Categorize the features based on the variable type of the features and the data it represents
- Numeric (Interval): Features which have equally spaced interval between unique values
- Categorical (Nominal): Features which have no intrinsic ordering to the unique values
- Ordinal: Features which have clear ordering but do not have equally spaced intervals between unique values

In [39]:
# categorizing features
num_cols = []
nom_cols = []
ord_cols = []

## Handling Outliers

In [ ]:
# check outlier summary only on numerical features
outlier_summary(X_train[num_cols], 1.5)

1. `Trimming`: if outliers' percentage < 5%
2. `Capping`: if outliers' percentage 5% - 15%
3. `None`: if outliers' percentage > 15%

In [ ]:
# check outlier details
check_outlier(X_train[num_cols], 1.5)

## Handling Missing Values

In [ ]:
# check missing values in train set
X_train.isna().sum()

In [ ]:
# check missing values in train target
y_train.isna().sum()

In [ ]:
# check missing values in test set
X_test.isna().sum()

In [ ]:
# check missing values in test target
y_test.isna().sum()

Great! There are no missing values in train nor test features and target

## Feature Selection

### Heatmap Correlation Matrix for Numerical Features

We look at the Spearman's correlation matrix to find out the relation between features and target

In [ ]:
# Heatmap Correlation Matrix
plt.figure(figsize=(15,10))

sns.heatmap(
    pd.concat([y_train, X_train[num_cols]], axis=1).corr('spearman'),
    annot=True, vmin=0, vmax=1, fmt='.2f', square=True)
plt.yticks(rotation=0)
plt.title('Heatmap for Numerical Features')

plt.show()


### Heatmap Correlation Matrix for Categorical Features

We look at the Spearman's correlation matrix to find out the relation between features and target

In [ ]:
# Heatmap Correlation Matrix
plt.figure(figsize=(15,10))

sns.heatmap(
    pd.concat([y_train, X_train[nom_cols]], axis=1).corr('spearman'),
    annot=True, vmin=0, vmax=1, fmt='.2f', square=True)
plt.yticks(rotation=0)
plt.title('Heatmap for Categorical Features')

plt.show()


- There are low spearman correlations amongst features and target
- However, since we do not really have any features that stand out, I'm going to use all features as predictors

### Heatmap Correlation Matrix for Ordinal Features

We look at the Spearman's correlation matrix to find out the relation between features and target

In [ ]:
# Heatmap Correlation Matrix
plt.figure(figsize=(15,10))

sns.heatmap(
    pd.concat([y_train, X_train[ord_cols]], axis=1).corr('spearman'),
    annot=True, vmin=0, vmax=1, fmt='.2f', square=True)
plt.yticks(rotation=0)
plt.title('Heatmap for Ordinal Features')

plt.show()


- There are quite a moderate correlations between the `pay_x` features and the target, at least higher than any in the numerical and categorical features
- However, we can also see that those features tend to be dependent amongst each other
- Still, since we do not really have any features that stand out, I'm going to use all features as predictors

### List of Features

These are the predictors we're going to use

In [ ]:
# print out list of predictors
print('Numerical Features:')
print(num_cols)
print('=' * 50)
print('Categorical Features:')
print(nom_cols)
print('=' * 50)
print('Ordinal Features:')
print(ord_cols)

### Create Pipeline

Create a pipeline based on how we would engineer the features, whether to scale or to encode
- We will create multiple pipeline for scaling because each features and each models require different procedures in which we handle it
- All categorical features will use one hot encoder
- Ordinal features which 

In [50]:
# create pipeline for standardization
std_pipe = Pipeline([
    ('std_scaler', StandardScaler())
])

# create pipeline for min max scaling
min_max_pipe = Pipeline([
    ('min_max_scaler', MinMaxScaler())
])

# create pipeline for normalizer
power_pipe = Pipeline([
    ('power_transformer', PowerTransformer())
])

# create pipeline for robust scaler
robust_pipe = Pipeline([
    ('robust_scaler', RobustScaler())
])

# create pipeline for categorical features
nom_pipe = Pipeline([
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])

### Create Column Transformer

Create a `ColumnTransformer` object based on the pipeline we have created

In [51]:
# create column transformer object using standard scaler
ct_std = ColumnTransformer([
    ('num', std_pipe, num_cols),
    ('nom', nom_pipe, nom_cols),
    ('ord', 'passthrough', ord_cols)
])

# create column transformer object using min max scaler
ct_mm = ColumnTransformer([
    ('num', min_max_pipe, num_cols),
    ('nom', nom_pipe, nom_cols),
    ('ord', 'passthrough', ord_cols)
])

# create column transformer object using power transform
ct_pt = ColumnTransformer([
    ('num', power_pipe, num_cols),
    ('nom', nom_pipe, nom_cols),
    ('ord', 'passthrough', ord_cols)
])

# create column transformer object using robust scaler
ct_rs = ColumnTransformer([
    ('num', robust_pipe, num_cols),
    ('nom', nom_pipe, nom_cols),
    ('ord', 'passthrough', ord_cols)
])


# 6. Model Definition

- Target: Predicting 


- Predictors: The features I'm going to use are

- Models: The Supervised Learning Algorithms I'm going to test

In [ ]:
# declare model

# 7. Model Training

In [ ]:
# create a dictionary of models
models = {}

## Cross Validating Base Models

Since the dataset is imbalance, we are going to evaluate our models based on **F1 Score**\
It's also important that we reduce the number of *False Negatives*, hence a high **Recall** is also important

In [60]:
# create a dict to store the cross validation scores
cv_results = {
    'models': [],
    'f1_score_mean': [],
    'f1_score_std': [],
    'recall_score_mean': [],
    'recall_score_std': []
} 

# loop over each each models and perform cross validation
for name, model in models.items():
    # fit the model
    model.fit(X_train, y_train)
    # get cross validation scores
    scores = cross_validate(
        model, X_train, y_train,
        scoring=['f1_weighted', 'recall'],
        cv=5
    )

    # store the cross validation scores
    cv_results['models'].append(name)
    cv_results['f1_score_mean'].append(scores['test_f1_weighted'].mean().round(2))
    cv_results['f1_score_std'].append(scores['test_f1_weighted'].std().round(4))
    cv_results['recall_score_mean'].append(scores['test_recall'].mean().round(2))
    cv_results['recall_score_std'].append(scores['test_recall'].std().round(4))

# create a dataframe from the dict
cv_results_df = pd.DataFrame(cv_results)

# 8. Model Evaluation

In [ ]:
# display the dataframe sorted by f1 score

## Hyperparameter Tuning


In [62]:
# create parameter grid

In [63]:
# create grid search object

In [ ]:
%%time

# perform grid search

In [ ]:
# print the best parameters

# print the best score

In [66]:
# assign the best estimator to the final model

Running this grid search resulted in:
- pass

## Evaluate Grid Search Results

In [67]:
# predict train set using the base model

# predict test set using the base model

# predict train set using the final model

# predict test set using the final model

In [68]:
# prepare target names for classification report

### Base Model Evaluation

In [ ]:
# create classification report for train set

# create classification report for test set

In [ ]:
# plot roc curve for train set
# calculate auc score for train set

# plot roc curve for test set
# calculate auc score for test set

### Final Model Evaluation

In [ ]:
# create classification report for train set

# create classification report for test set

In [ ]:
# plot roc curve for train set
# calculate auc score for train set

# plot roc curve for test set
# calculate auc score for test set

#### Analysis
1. pass

## Save The Final Model

In [ ]:
# prepare directory for saving model
model_dir = 'models'
model_name = ''

# create directory if it does not exist
Path(model_dir).mkdir(parents=True, exist_ok=True)

# save model

# 9. Model Inference

## Load The Model

In [74]:
# model location
model_dir = 'models'
model_name = ''
model_path = Path(model_dir, model_name)

# load model
final_svm = joblib.load(model_path)

In [ ]:
# display inference set
df_inf.head()

## Inferencing

In [ ]:
%%time

# predict inference set using the final model
y_pred_inf_svm = final_svm.predict(df_inf)

In [ ]:
# create dataframe with predictions
df_inf['pred_svm'] = y_pred_inf_svm

# display inference set
df_inf

Model successfully run on inference dataset

# 10. Conclusion

## On EDA
- pass

## On Modeling
- pass

## Implication
- pass

## Future Improvement
- pass